# ML model exposed through API

In [ ]:
import os
os.getcwd()
os.chdir("C:\\Users\\agadiya\\Desktop\\Convergence - Launch your Ideas")
import pandas as pd

In [2]:
response=pd.read_csv("response.csv")

In [3]:
response.head()

,health_app_code,ques_1_response,ques_2_response,ques_3_response,ques_4_response,ques_5_response,ques_6_response,ques_7_response,ques_8_response,ques_9_response,ques_10_response,device_class
0,1,0,0,0,0,0,0,1,0,0,1,2
1,2,0,0,0,0,0,1,0,1,0,0,3
2,3,1,0,1,1,0,1,1,1,0,0,1
3,4,0,1,0,0,1,1,1,1,1,0,3
4,5,1,0,1,1,0,1,0,1,1,1,3


In [4]:
X = response.iloc[:, 1:11].values
X[1]

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0], dtype=int64)

In [5]:
y = response.iloc[:, 11].values

In [6]:
y


array([2, 3, 1, ..., 1, 3, 3], dtype=int64)

In [7]:
# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

C:\Users\agadiya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)


C:\Users\agadiya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [9]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [10]:
# save the model to disk
import pickle
filename = 'model.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [11]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.3541666666666667


In [12]:
pred = loaded_model.predict(X_test)
print(pred)

[1 3 2 2 1 3 1 1 1 1 1 2 3 2 1 3 2 1 2 2 2 3 3 2 3 1 1 1 2 2 2 3 2 1 2 2 3
 3 3 1 3 3 3 1 3 3 3 2 3 1 2 3 1 3 1 3 1 1 2 2 2 3 3 3 3 2 3 1 2 1 3 3 3 1
 2 2 2 1 2 2 2 3 3 2 2 1 3 1 1 2 1 3 3 3 3 1 1 1 2 3 3 3 2 1 1 1 3 1 1 3 2
 3 2 1 2 1 2 1 1 2 3 3 1 3 1 1 2 2 2 1 1 1 2 1 2 2 1 2 1 3 2 2 1 1 2 3 2 2
 2 1 1 1 2 2 1 1 3 1 2 3 2 1 2 1 3 3 3 2 2 3 3 3 2 2 2 2 3 2 3 1 1 3 1 3 2
 2 2 3 2 3 1 3 2 2 2 3 1 1 2 2 1 1 2 1 1 3 2 2 1 1 1 3 2 3 1 3 1 1 1 1 3 3
 1 2 1 1 2 2 3 2 2 1 1 2 2 2 3 1 1 3]


In [13]:
pred1=loaded_model.predict([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
pred1

array([3], dtype=int64)

In [14]:
import json,requests

In [ ]:
import numpy as np
from flask import Flask, abort, jsonify, request
import pickle
from flask import Response


my_random_forest = pickle.load(open(filename, 'rb'))

app= Flask(__name__)

@app.route('/api/make_predict',methods=['POST'])

def make_predict():
    #all kinds of error checking should go here
    data = request.get_json(force=True)
    #convert our json to numpy array
    predict_request = [data['ques_1_response'],data['ques_2_response'],data['ques_3_response'],data['ques_4_response'],data['ques_5_response'],data['ques_6_response'],data['ques_7_response'],data['ques_8_response'],data['ques_9_response'],data['ques_10_response']]
    #predict_request1=np.array(predict_request)
    predict_request1 = [predict_request]
    #np array goes into random forest
    y_hat=my_random_forest.predict(predict_request1)
    #return our prediction
    output=[y_hat[0]]
    prediction_series = list(pd.Series(y_hat[0]))
    final_predictions = pd.DataFrame(list(prediction_series))
    responses = jsonify(predictions=final_predictions.to_json(orient="records"))
   # responses.status_code = 200
    return (responses)

if __name__ == '__main__':
    app.run(port = 8000)

 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Dec/2018 13:59:47] "OPTIONS /api/make_predict?ques_1_response=0&ques_2_response=1&ques_3_response=1&ques_4_response=0&ques_5_response=1&ques_6_response=0&ques_7_response=1&ques_8_response=0&ques_9_response=1&ques_10_response=0 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2018 14:01:43] "OPTIONS /api/make_predict?ques_1_response=1&ques_2_response=1&ques_3_response=1&ques_4_response=1&ques_5_response=1&ques_6_response=0&ques_7_response=1&ques_8_response=0&ques_9_response=1&ques_10_response=1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2018 14:01:44] "POST /api/make_predict?ques_1_response=1&ques_2_response=1&ques_3_response=1&ques_4_response=1&ques_5_response=1&ques_6_response=0&ques_7_response=1&ques_8_response=0&ques_9_response=1&ques_10_response=1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2018 14:13:23] "OPTIONS /api/make_predict?ques_1_response=1&ques_2_response=0&ques_3_response=0&ques_4_response=1&ques_5_response=1&ques_6